In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from ResidualAutoEncoder import ResidualAutoencoder, wandb_sweep
from ResidualAutoEncoder import Criterion

In [2]:
def main():
    # Hyperparameters
    batch_size = 16
    learning_rate = 0.001
    epochs = 15
    
    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Data transformation and loading
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
        transforms.Resize((128, 128))
    ])
    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=3, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    

    
    # Model, loss function, and optimizer
    model = ResidualAutoencoder(num_blocks=3, block_depth=2, bottleneck_dim=128, channels=3, device=device)
    criterion = Criterion(model=model)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Train the model
    model.train_harness(model, train_loader, criterion, optimizer, epochs)
    
    torch.save(model.state_dict(), 'models/residual_autoencoder.pth')
    # Evaluate the model
    inputs, outputs = model.evaluate_harness(model, test_loader, device)
    outputs = unnormalize(outputs)
    # Visualize the results
    num_images = 10
    fig, axes = plt.subplots(2, num_images, figsize=(15, 4))
    for i in range(num_images):
        axes[0, i].imshow(inputs[i].permute(1, 2, 0))
        axes[0, i].axis('off')
        axes[1, i].imshow(outputs[i].permute(1, 2, 0))
        axes[1, i].axis('off')
    
    plt.show()
    
def unnormalize(tensor):
    return tensor * 0.5 + 0.5

if __name__ == "__main__":
    main()

Files already downloaded and verified
Files already downloaded and verified


c:\Users\Arjun\anaconda3\envs\PyTorch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Arjun\anaconda3\envs\PyTorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


TypeError: int is not an Optimizer

In [2]:
import wandb
wandb.login()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vinmed-arjun (subarjun). Use `wandb login --relogin` to force relogin


True

In [4]:
sweep_config = {
    'method': 'random',  # or 'grid' or 'bayes'
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'num_blocks': {
            'values': [2, 3, 4]
        },
        'block_depth': {
            'values': [2, 3]
        },
        'bottleneck_dim': {
            'values': [64, 128, 256]
        },
        'asym_block': {
            'values': [1]
        },
        'asym_depth': {
            'values': [1]
        },
        'learning_rate': {
        'distribution': 'q_log_uniform_values',
        'q' : 1e-6,
        'max': 0.01,
        'min': 1e-6,
        },
        'l1_lambda': {
        'distribution': 'q_log_uniform_values',
        'q' : 1e-15,
        'max': 1e-6,
        'min': 1e-15,
        },
        'l1_lambda_bottleneck': {
        'distribution': 'q_log_uniform_values',
        'q' : 1e-10,
        'max': 1e-4,
        'min': 1e-10,
        },
        'lambda_kl': {
        'distribution': 'q_log_uniform_values',
        'q' : 1e-10,
        'max': 1e-4,
        'min': 1e-10,
        },
        'lambda_tv': {
        'distribution': 'q_log_uniform_values',
        'q' : 1e-19,
        'max': 1e-8,
        'min': 1e-19,
        },
        'lambda_perceptual': {
        'distribution': 'q_log_uniform_values',
        'q' : 1e-6,
        'max': 1e-1,
        'min': 1e-6,
        },
        'lambda_mse': {
        'distribution': 'q_log_uniform_values',
        'q' : 1e1,
        'min': 1e1,
        'max': 1e2,
        },
        'lambda_ssim': {
        'distribution': 'q_log_uniform_values',
        'q' : 1e-6,
        'max': 1e-1,
        'min': 1e-6,
        },
        'lambda_psnr': {
        'distribution': 'q_log_uniform_values',
        'q' : 1e-19,
        'max': 1e-8,
        'min': 1e-19,
        },
        'dropout': {
            'values': [0.2, 0.3, 0.4]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="residual-autoencoder")


Create sweep with ID: z9gikt4e
Sweep URL: https://wandb.ai/subarjun/residual-autoencoder/sweeps/z9gikt4e


In [5]:
wandb.agent(sweep_id, wandb_sweep, count=25)

wandb: Agent Starting Run: kr9dgixv with config:
wandb: 	asym_block: 1
wandb: 	asym_depth: 1
wandb: 	block_depth: 2
wandb: 	bottleneck_dim: 256
wandb: 	dropout: 0.4
wandb: 	l1_lambda: 2.497098e-09
wandb: 	l1_lambda_bottleneck: 1.91e-08
wandb: 	lambda_kl: 3.5465000000000003e-06
wandb: 	lambda_mse: 130
wandb: 	lambda_perceptual: 4.2e-05
wandb: 	lambda_psnr: 8.4803911665e-09
wandb: 	lambda_ssim: 0.0008849999999999999
wandb: 	lambda_tv: 2.1398006735e-09
wandb: 	learning_rate: 0.001109
wandb: 	num_blocks: 4
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Files already downloaded and verified
Files already downloaded and verified


c:\Users\Arjun\anaconda3\envs\PyTorch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Arjun\anaconda3\envs\PyTorch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\Arjun\anaconda3\envs\PyTorch\Lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
 

Epoch [1/25], Loss: 93.9509, recon_loss: 0.0305, val_loss: 1432.1378 learning rate: 0.001109
Epoch [2/25], Loss: 91.5569, recon_loss: 0.0172, val_loss: 917.6717 learning rate: 0.001109
Epoch [3/25], Loss: 91.3299, recon_loss: 0.0190, val_loss: 860.1141 learning rate: 0.001109
Epoch [4/25], Loss: 91.1935, recon_loss: 0.0186, val_loss: 784.1443 learning rate: 0.001109
Epoch [5/25], Loss: 91.1371, recon_loss: 0.0153, val_loss: 844.8107 learning rate: 0.001109
Epoch [6/25], Loss: 91.1039, recon_loss: 0.0141, val_loss: 709.7896 learning rate: 0.001109
Epoch [7/25], Loss: 91.0191, recon_loss: 0.0114, val_loss: 683.7635 learning rate: 0.001109
Epoch [8/25], Loss: 91.0060, recon_loss: 0.0166, val_loss: 672.7143 learning rate: 0.001109
Epoch [9/25], Loss: 91.0112, recon_loss: 0.0143, val_loss: 670.0866 learning rate: 0.001109
